In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import matplotlib.lines as mlines
import matplotlib.patches as patches
import warnings
from pandas.core.common import SettingWithCopyWarning # 1.4.4
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter("ignore", UserWarning)

## **Estadísticas generales (visualizaciones de linea de tiempo)** 

In [ ]:
### Obtener datos 
nombre_archivo = os.path.join('..', 'datos', 'datos_procesados', 'nodos.csv')
datos = pd.read_csv(nombre_archivo)

# Procesamiento de datos
datos['fecha_toma'] = pd.to_datetime(datos['fecha_toma'])
totales = datos.groupby(pd.Grouper(key='fecha_toma', freq='M')).count().rename(columns={'id_muestra':'total'})['total']

# Agrupado por meses para obtener porcentaje de resistencia
b0 = datos[datos['resistencia_bool']=='Con resistencia'].groupby([pd.Grouper(key='fecha_toma', freq='M'),'resistencia_bool']).count().reset_index().rename(
columns={'id_muestra':'pacientes_con_resistencia'})[['fecha_toma','pacientes_con_resistencia']]
b = b0.merge(totales,on='fecha_toma')
b.insert(len(b.columns),'porcentaje',b['pacientes_con_resistencia']*100/b['total'])
b = b.round(1)

b = b.astype({'fecha_toma':str})
b['fecha_toma'] = b['fecha_toma'].str.slice(stop=-3)
b.set_index('fecha_toma')
b.head()

In [ ]:

# Agrupado por meses para sacar porcentaje de resistencia por tipo de ARV
b1 = datos[datos['resistencia_bool']=='Con resistencia'].groupby([pd.Grouper(key='fecha_toma', freq='M'),'resistencia']).count().reset_index().rename(
    columns={'id_muestra':'pacientes_con_resistencia'})[['fecha_toma','resistencia','pacientes_con_resistencia']]
bARV = b1.merge(totales,on='fecha_toma')
bARV.insert(len(bARV.columns),'porcentaje',bARV['pacientes_con_resistencia']*100/bARV['total'])
bARV=bARV.round(1)
bARV = bARV.astype({'fecha_toma':str})
bARV['fecha_toma'] = bARV['fecha_toma'].str.slice(stop=-3)
bARV.sort_values("resistencia", ascending = True)
bARV.head()

#### __Cambio en la proporción de participantes con resistencia a antirretrovirales a través del tiempo (Clase ARV)__
<img src = "../datos/imagenes/participantes_resistencia_tiempo_claseARV.png" width= "700"/>

In [ ]:
# Crear marco de figura 
fig, ax = plt.subplots(figsize=(12, 5))
sns.despine(top = True, right = True, left = True, bottom = True)

# Propiedades de cuadrícula
sns.set_style("whitegrid")
sns.set_context("notebook")
plt.grid(axis = 'y', color = "#7D7D7D", linestyle = "--", linewidth = 0.7)
plt.grid(visible = False, axis = 'x')

# Dibujar gráfica 
colores = ["#495CD9", "#F0C859", "#FF302B", "#E57351", "#97302B"]
sns.lineplot(data = bARV, x = "fecha_toma", y = "porcentaje", hue= "resistencia", hue_order=['Compleja', 'INSTI', 'NNRTI', 'NRTI', 'PI'], 
             palette = colores, linewidth = 0.7)

# Editar eje X para que no muestre todas las fechas 
fechas_especificas = ["2020-01", "2021-01", "2022-01", "2023-01"]
cifras_especificas = [0, 5, 10, 15]
plt.gca().set_xticks(fechas_especificas)
plt.gca().set_yticks(cifras_especificas) 
plt.xticks(ticks = ["2020-01", "2021-01", "2022-01", "2023-01"], labels = ["ene/2020", "ene/2021", "ene/2022", "ene/2023" ])

# Editar marcadores, nombre y posición de leyenda 
handles, labels = plt.gca().get_legend_handles_labels()
for handle in handles: 
    handle.set_marker('o')
    handle.set_linewidth(0)
    handle.set_markersize(12)

plt.legend(title = "Resistencia a antirretrovirales", frameon = False, loc = "lower center", ncol = 5, borderaxespad = -8) 

#  Editar leyendas de ejes 
ax.set_xlabel("Fecha", fontsize = 14, labelpad = 15, weight = "bold")
ax.set_ylabel("Resistencia (%)", fontsize = 14, labelpad = 15, weight = "bold")

# Crear rectangulo (fechas sin valores por covid)
rect = patches.Rectangle ((8,0), 1, 19, linewidth = 0, edgecolor = 'none', facecolor = 'lightgray')
plt.gca().add_patch(rect)

plt.show()

#### __Cambio en la proporción de participantes con resistencia a antirretrovirales a través del tiempo (General)__
<img src = "../datos/imagenes/participantes_resistencia_tiempo_general.png" width= "700"/>

In [ ]:
# Crear marco de figura 
fig, ax = plt.subplots(figsize=(12, 5))
sns.despine(top = True, right = True, left = True, bottom = True)

# Propiedades de cuadrícula
sns.set_style("whitegrid")
sns.set_context("notebook")
plt.grid(axis = 'y', color = "#7D7D7D", linestyle = "--", linewidth = 0.7)
plt.grid(visible = False, axis = 'x')

# Dibujar gráfica 
sns.lineplot(data = b, x = "fecha_toma", y = "total", color = "#A834F6", marker = "o", linewidth = 0.7)
plt.axhline(y = 70, color = "black", linewidth = 0.5)

# Editar eje X para que no muestre todas las fechas 
fechas_especificas = ["2020-01", "2021-01", "2022-01", "2023-01"]
cifras_especificas = [100, 150, 200, 250, 300, 350]
plt.gca().set_xticks(fechas_especificas)
plt.gca().set_yticks(cifras_especificas) 

# Cambiar formato de fecha
plt.xticks(ticks = ["2020-01", "2021-01", "2022-01", "2023-01"], labels = ["ene/2020", "ene/2021", "ene/2022", "ene/2023"])

#  Editar leyendas de ejes 
ax.set_xlabel("Fecha", fontsize = 14, labelpad = 15, weight = "bold")
ax.set_ylabel("Resistencia (%)", fontsize = 14, labelpad = 15, weight = "bold")

# Crear rectangulo (fechas sin valores por covid)
rect = patches.Rectangle ((8,70), 1, 350, linewidth = 0, edgecolor = 'none', facecolor = 'lightgray')
plt.gca().add_patch(rect)

plt.show()

## **Red (visualizaciones de linea de tiempo)** 

In [ ]:
# Procesamiento de datos 

columnas = ['id_muestra', 'fecha_toma', 'cluster_id']
for column in datos:
    datos[column] = datos[column].replace(np.nan, None)

def ifelse(x, check, yes, no):
    if x == check:
        res = yes
    else:
        res = no
    return(res)

datos['en_cluster'] = datos['cluster_id'].apply(lambda x: ifelse(x, 0, 0, 1))
datos['fecha_toma'] = pd.to_datetime(datos['fecha_toma'], format="%Y/%m/%d")
datos['anio_mes'] = datos['fecha_toma'].dt.strftime('%Y-%m')

datos_personas = (datos.groupby('anio_mes').agg({'en_cluster': ['count', 'sum']}).droplevel(0, axis=1))

datos_personas.reset_index(inplace=True)
datos_personas.rename(columns={'sum': 'tooltip_personas_en_cong_no_acum'}, inplace=True)
datos_personas.rename(columns={'count': 'personas_tot'}, inplace=True)
datos_personas.reset_index(inplace=True)

datos_personas['tooltip_personas_en_cong_acum'] = datos_personas['tooltip_personas_en_cong_no_acum'].cumsum()
datos_personas['personas_acumuladas'] = datos_personas['personas_tot'].cumsum()

datos_personas['porcentaje_de_personas_cong_acum'] = round((datos_personas['tooltip_personas_en_cong_acum']/datos_personas['personas_acumuladas'])*100,2)
datos_personas['porcentaje_de_personas_cong_no_acum'] = round((datos_personas['tooltip_personas_en_cong_no_acum']/datos_personas['personas_tot'])*100,2)

columnas_finales = ['anio_mes', 'tooltip_personas_en_cong_acum', 'porcentaje_de_personas_cong_acum',
                    'tooltip_personas_en_cong_no_acum', 'porcentaje_de_personas_cong_no_acum']
datos_personas_final = datos_personas[columnas_finales]

In [ ]:
datos_personas_final.head()

#### __Participantes que conforman un conglomerado (mensual)__
<img src = "../datos/imagenes/participantes_conglomerado_tiempo_mensual.png" width= "700"/>

In [ ]:
# Crear marco de figura 
fig, ax = plt.subplots(figsize=(12, 5))
sns.despine(top = True, right = True, left = True, bottom = True)

# Propiedades de cuadrícula
sns.set_style("whitegrid")
sns.set_context("notebook")
plt.grid(axis = 'y', color = "#7D7D7D", linestyle = "--", linewidth = 0.7)
plt.grid(visible = False, axis = 'x')

# Dibujar gráfica 
sns.lineplot(data = datos_personas_final, x = "anio_mes", y = "porcentaje_de_personas_cong_no_acum", color = "#3221ED", marker = "o", linewidth = 0.7)
plt.axhline(y = 40, color = "black", linewidth = 0.5)

# Editar eje X para que no muestre todas las fechas 
fechas_especificas = ["2020-01", "2021-01", "2022-01", "2023-01"]
plt.gca().set_xticks(fechas_especificas)

# Cambiar formato de fecha
plt.xticks(ticks = ["2020-01", "2021-01", "2022-01", "2023-01"], labels = ["ene/2020", "ene/2021", "ene/2022", "ene/2023"])
plt.yticks(ticks = [45, 50, 55], labels = ["45%", "50%", "55%"])

#  Editar leyendas de ejes 
ax.set_xlabel("Fecha", fontsize = 14, labelpad = 15, weight = "bold")
ax.set_ylabel("Cantidad de participantes", fontsize = 14, labelpad = 15, weight = "bold")

# Crear rectangulo (fechas sin valores por covid)
rect = patches.Rectangle ((8,40), 1, 55, linewidth = 0, edgecolor = 'none', facecolor = 'lightgray')
plt.gca().add_patch(rect)

plt.show()

#### __Participantes que conforman un conglomerado (acumulado)__
<img src = "../datos/imagenes/participantes_conglomerado_tiempo_acumulado.png" width= "700"/>

In [ ]:
# Crear marco de figura 
fig, ax = plt.subplots(figsize=(12, 5))
sns.despine(top = True, right = True, left = True, bottom = True)

# Propiedades de cuadrícula
sns.set_style("whitegrid")
sns.set_context("notebook")
plt.grid(axis = 'y', color = "#7D7D7D", linestyle = "--", linewidth = 0.7)
plt.grid(visible = False, axis = 'x')

# Dibujar gráfica 
sns.lineplot(data = datos_personas_final, x = "anio_mes", y = "porcentaje_de_personas_cong_acum", color = "#3221ED", marker = "o", linewidth = 0.7)
plt.axhline(y = 48.5, color = "black", linewidth = 0.5)

# Editar eje X para que no muestre todas las fechas 
fechas_especificas = ["2020-01", "2021-01", "2022-01", "2023-01"]
plt.gca().set_xticks(fechas_especificas)

# Cambiar formato de fecha
plt.xticks(ticks = ["2020-01", "2021-01", "2022-01", "2023-01"], labels = ["ene/2020", "ene/2021", "ene/2022", "ene/2023"])
plt.yticks(ticks = [49, 50, 51, 52, 53], labels = ["49%", "50%", "52%", "52%", "53%"])

#  Editar leyendas de ejes 
ax.set_xlabel("Fecha", fontsize = 14, labelpad = 15, weight = "bold")
ax.set_ylabel("Cantidad de participantes", fontsize = 14, labelpad = 15, weight = "bold")

# Crear rectangulo (fechas sin valores por covid)
rect = patches.Rectangle ((8,48.5), 1, 53, linewidth = 0, edgecolor = 'none', facecolor = 'lightgray')
plt.gca().add_patch(rect)

plt.tight_layout
plt.show()

In [ ]:
# Procesamiento de datos 
datos_conglomerados = datos[datos['cluster_id'] != 0]
datos_conglomerados['fecha_toma'] = pd.to_datetime(datos_conglomerados['fecha_toma'], format="%Y/%m/%d")
datos_conglomerados['anio_mes'] = datos_conglomerados['fecha_toma'].dt.strftime('%Y-%m')
datos_conglomerados.sort_values(by='fecha_toma', inplace = True)

meses = datos_conglomerados['anio_mes'].unique()
df_fech_cong = datos_conglomerados[['anio_mes', 'cluster_id']]
lista_conglomerados = []
mensual = {}
for m in meses:
    df_tmp = df_fech_cong[df_fech_cong['anio_mes'] == m]
    conglomerados = df_tmp['cluster_id'].unique()
    conglomerados_mes = list(set(conglomerados).difference(lista_conglomerados))
    mensual[m] = len(conglomerados_mes)
    lista_conglomerados = lista_conglomerados + conglomerados_mes

datos_mensual = pd.DataFrame.from_dict(mensual, orient='index').reset_index()
datos_mensual.columns = ['anio_mes', 'num_conglomerados_no_acum']
datos_mensual['num_conglomerados_acum'] = datos_mensual['num_conglomerados_no_acum'].cumsum()   

In [ ]:
datos_mensual.head()

#### __Número de conglomerados en el tiempo (mensual)__
<img src = "../datos/imagenes/numero_conglomerados_tiempo_mensual.png" width = "700"/>

In [ ]:
# Crear marco de figura 
fig, ax = plt.subplots(figsize=(12, 5))
sns.despine(top = True, right = True, left = True, bottom = True)

# Propiedades de cuadrícula
sns.set_style("whitegrid")
sns.set_context("notebook")
plt.grid(axis = 'y', color = "#7D7D7D", linestyle = "--", linewidth = 0.7)
plt.grid(visible = False, axis = 'x')

# Dibujar gráfica 
sns.lineplot(data = datos_mensual, x = "anio_mes", y = "num_conglomerados_no_acum", color = "#A834F6", marker = "o", linewidth = 0.7)
plt.axhline(y = 0, color = "black", linewidth = 0.5)

# Editar eje X para que no muestre todas las fechas 
fechas_especificas = ["2020-01", "2021-01", "2022-01", "2023-01"]
plt.gca().set_xticks(fechas_especificas)

# Cambiar formato de fecha
plt.xticks(ticks = ["2020-01", "2021-01", "2022-01", "2023-01"], labels = ["ene/2020", "ene/2021", "ene/2022", "ene/2023"])
plt.yticks(ticks = [20, 40, 60, 80, 100])

#  Editar leyendas de ejes 
ax.set_xlabel("Fecha", fontsize = 14, labelpad = 15, weight = "bold")
ax.set_ylabel("Cantidad de conglomerados", fontsize = 14, labelpad = 15, weight = "bold")

# Crear rectangulo (fechas sin valores por covid)
rect = patches.Rectangle ((8,0), 1, 100, linewidth = 0, edgecolor = 'none', facecolor = 'lightgray')
plt.gca().add_patch(rect)

plt.show()

#### __Número de conglomerados en el tiempo (acumulado)__
<img src = "../datos/imagenes/numero_conglomerados_tiempo_acumulado.png" width = "700"/>


In [ ]:
# Crear marco de figura 
fig, ax = plt.subplots(figsize=(12, 5))
sns.despine(top = True, right = True, left = True, bottom = True)

# Propiedades de cuadrícula
sns.set_style("whitegrid")
sns.set_context("notebook")
plt.grid(axis = 'y', color = "#7D7D7D", linestyle = "--", linewidth = 0.7)
plt.grid(visible = False, axis = 'x')

# Dibujar gráfica 
sns.lineplot(data = datos_mensual, x = "anio_mes", y = "num_conglomerados_acum", color = "#A834F6", marker = "o", linewidth = 0.7)
plt.axhline(y = 0, color = "black", linewidth = 0.5)

# Editar eje X para que no muestre todas las fechas 
fechas_especificas = ["2020-01", "2021-01", "2022-01", "2023-01"]
plt.gca().set_xticks(fechas_especificas)

# Cambiar formato de fecha
plt.xticks(ticks = ["2020-01", "2021-01", "2022-01", "2023-01"], labels = ["ene/2020", "ene/2021", "ene/2022", "ene/2023"])
plt.yticks(ticks = [500, 1000, 1500])

#  Editar leyendas de ejes 
ax.set_xlabel("Fecha", fontsize = 14, labelpad = 15, weight = "bold")
ax.set_ylabel("Cantidad de conglomerados", fontsize = 14, labelpad = 15, weight = "bold")

# Crear rectangulo (fechas sin valores por covid)
rect = patches.Rectangle ((8,0), 1, 1600, linewidth = 0, edgecolor = 'none', facecolor = 'lightgray')
plt.gca().add_patch(rect)

plt.show()